In [ ]:
# ============================================================
# Google Colab  ➜  Copy Shared Drive Folder To Your Drive
# ============================================================

# Install/update required Google API and helper libraries
!pip -q install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib humanize

from google.colab import auth, drive as gdrive_mount
auth.authenticate_user()               # Authenticate user with Google account for API access
gdrive_mount.mount('/content/drive')  # Mount Google Drive to /content/drive

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import re, time, humanize
from collections import deque

# Initialize Drive API client
service = build('drive', 'v3', cache_discovery=False)

In [2]:
# ====== CONFIGURATION ======
# Paste the shared folder URL here (public or shared with you)
SRC_FOLDER_URL = "https://drive.google.com/drive/folders/YOUR_SHARED_FOLDER_ID"
# Destination folder path inside your MyDrive; should start with '/'
DEST_PATH = "/DownloadsColab"
# ===========================

In [ ]:
def extract_folder_id(url):
    # Extract Google Drive folder ID from URL using regex patterns
    patt = [r'/folders/([a-zA-Z0-9_-]+)', r'id=([a-zA-Z0-9_-]+)']
    for p in patt:
        m = re.search(p, url)
        if m: return m.group(1)
    raise ValueError("Cannot find folder ID in URL.")

def list_children(fid):
    # List all files and folders inside a Drive folder, handling pagination
    q = f"'{fid}' in parents and trashed=false"
    children = []
    page = None
    while True:
        resp = service.files().list(
            q=q, fields="nextPageToken, files(id,name,mimeType,size)",
            supportsAllDrives=True, includeItemsFromAllDrives=True,
            corpora="allDrives", pageToken=page).execute()
        children.extend(resp.get('files', []))
        page = resp.get('nextPageToken')
        if not page: break
    return children

def ensure_folder(path):
    # Ensure folder hierarchy exists in Drive; create folders if needed
    parent = 'root'
    for part in path.strip('/').split('/'):
        q = (f"name='{part}' and '{parent}' in parents and "
             "mimeType='application/vnd.google-apps.folder' and trashed=false")
        res = service.files().list(q=q, fields="files(id)", supportsAllDrives=True).execute()
        if res['files']:
            parent = res['files'][0]['id']  # Folder exists, update parent
        else:
            # Create folder and update parent
            meta = {'name': part, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [parent]}
            parent = service.files().create(body=meta, fields='id').execute()['id']
    return parent  # Return ID of deepest folder

def copy_file(src_id, name, dest_id):
    # Copy a file from source folder to destination folder in Drive
    body = {'name': name, 'parents':[dest_id]}
    new_meta = service.files().copy(fileId=src_id, body=body, supportsAllDrives=True).execute()
    size = int(new_meta.get('size', '0'))
    return size

def fmt_bytes(n):
    # Format byte count into human-readable string
    return humanize.naturalsize(n, binary=True)

# Print configuration info
print(f"🔗 Source: {SRC_FOLDER_URL}")
print(f"📁 Destination: {DEST_PATH}")

# Extract IDs and ensure destination folder exists
src_root_id = extract_folder_id(SRC_FOLDER_URL)
dst_root_id = ensure_folder(DEST_PATH)

print("\n🔍 Scanning source folder tree …")
folders_map = {src_root_id: dst_root_id}
files_queue = []
total_size = 0

# Queue for breadth-first traversal of folder tree
dq = deque([(src_root_id, dst_root_id)])
while dq:
    sid, did = dq.popleft()
    for item in list_children(sid):
        if item['mimeType'] == 'application/vnd.google-apps.folder':
            # Create subfolder inside destination and queue for scan
            nid = service.files().create(
                body={'name': item['name'], 'mimeType': item['mimeType'], 'parents':[did]},
                fields='id').execute()['id']
            folders_map[item['id']] = nid
            dq.append((item['id'], nid))
        else:
            files_queue.append((item, did))
            if 'size' in item:
                total_size += int(item['size'])

total_files = len(files_queue)
print(f"✅ Found {total_files} files in {len(folders_map)} folders\n")

# Copy files one by one, printing progress
copied, failed = 0, 0
for idx, (meta, parent_id) in enumerate(files_queue, 1):
    name = meta['name']
    size = int(meta.get('size', '0'))
    print(f"[{idx}/{total_files}] 📄 {name}  ({fmt_bytes(size) if size else 'Unknown size'})")
    try:
        copy_file(meta['id'], name, parent_id)
        copied += 1
    except HttpError as e:
        print(f"   ❌ Error: {e}")
        failed += 1

print(f"\n🎉 Done! Copied: {copied} files • Failed: {failed} files")
print(f"📂 All files saved to: {DEST_PATH}")
print(f"📦 Total size copied: {fmt_bytes(total_size)}")